In [1]:
# ===========================================
# PARAMÈTRES DE CONFIGURATION
# ===========================================

# Date de fin d'analyse (format YYYY-MM-DD)
# Si None, utilise la date actuelle
DATE_FIN = None  # Exemple: "2024-12-31" ou None pour aujourd'hui

# Durée d'analyse (remonte dans le temps depuis DATE_FIN)
DUREE_JOURS = 1

# Intervalle des données
# Options: 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M
INTERVAL = '1m'

# Période de la moyenne mobile (en nombre de périodes)
MA_PERIODE = 30

print(f"Configuration:")
print(f"- Date de fin: {DATE_FIN if DATE_FIN else 'Aujourd\'hui'}")
print(f"- Durée d'analyse: {DUREE_JOURS} jours")
print(f"- Intervalle: {INTERVAL}")
print(f"- Moyenne mobile: {MA_PERIODE} périodes")
print()

# ===========================================
# RÉCUPÉRATION DES DONNÉES
# ===========================================

import pandas as pd
import requests
from datetime import datetime, timedelta
import time

print("Récupération des données Bitcoin...")

# Fonction pour récupérer des données par batch
def get_historical_data(symbol='BTCUSDT', interval=INTERVAL, start_time=None, days_back=DUREE_JOURS, end_date=None):
    """
    Récupère les données selon les paramètres configurés
    """
    all_data = []
    limit = 1000  # Maximum autorisé par Binance
    
    # Déterminer les dates de début et fin
    if end_date:
        end_time = int(pd.to_datetime(end_date).timestamp() * 1000)
        start_time = int((pd.to_datetime(end_date) - pd.Timedelta(days=days_back)).timestamp() * 1000)
    else:
        end_time = int(datetime.now().timestamp() * 1000)
        if start_time is None:
            start_time = int((datetime.now() - pd.Timedelta(days=days_back)).timestamp() * 1000)
    
    current_time = start_time
    
    # Calcul de l'incrément temporel selon l'intervalle
    interval_ms = {
        '1m': 60 * 1000,
        '3m': 3 * 60 * 1000,
        '5m': 5 * 60 * 1000,
        '15m': 15 * 60 * 1000,
        '30m': 30 * 60 * 1000,
        '1h': 60 * 60 * 1000,
        '2h': 2 * 60 * 60 * 1000,
        '4h': 4 * 60 * 60 * 1000,
        '6h': 6 * 60 * 60 * 1000,
        '8h': 8 * 60 * 60 * 1000,
        '12h': 12 * 60 * 60 * 1000,
        '1d': 24 * 60 * 60 * 1000,
        '3d': 3 * 24 * 60 * 60 * 1000,
        '1w': 7 * 24 * 60 * 60 * 1000,
        '1M': 30 * 24 * 60 * 60 * 1000
    }
    
    time_increment = interval_ms.get(interval, 60 * 1000)  # Défaut: 1 minute
    
    batch_count = 0
    while current_time < end_time:
        batch_count += 1
        print(f"Batch {batch_count} ({symbol}): Récupération depuis {datetime.fromtimestamp(current_time/1000).strftime('%Y-%m-%d %H:%M')}")
        
        # Construction de l'URL avec startTime et endTime
        url = f"https://api.binance.com/api/v3/klines?symbol={symbol}&interval={interval}&limit={limit}&startTime={current_time}&endTime={end_time}"
        
        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()
            
            if not data:
                break
                
            all_data.extend(data)
            
            # Mettre à jour current_time avec le timestamp de la dernière bougie + intervalle
            last_timestamp = data[-1][0]
            current_time = last_timestamp + time_increment
            
            # Pause pour respecter les limites de taux
            time.sleep(0.2)
            
        except requests.exceptions.RequestException as e:
            print(f"Erreur lors de la requête: {e}")
            break
    
    print(f"Total de {len(all_data)} périodes ({interval}) récupérées pour {symbol}")
    return all_data

# Récupération des données Bitcoin selon les paramètres configurés
print(f"=== Récupération Bitcoin (BTCUSDT) - Intervalle {INTERVAL} ({DUREE_JOURS} jours) ===")
if DATE_FIN:
    print(f"Période: {DUREE_JOURS} jours jusqu'au {DATE_FIN}")
else:
    print(f"Période: {DUREE_JOURS} jours jusqu'à aujourd'hui")

btc_response = get_historical_data('BTCUSDT', interval=INTERVAL, days_back=DUREE_JOURS, end_date=DATE_FIN)

# Fonction pour créer un DataFrame
def create_dataframe(data, symbol_name):
    df = pd.DataFrame(data, columns=[
        'timestamp', 'open', 'high', 'low', 'close', 'volume',
        'close_time', 'quote_asset_volume', 'number_of_trades',
        'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
    ])
    
    # Conversion des types
    for col in ['open', 'high', 'low', 'close', 'volume']:
        df[col] = df[col].astype(float)
    
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    
    # Affichage des informations adaptées à l'intervalle
    print(f"\nDonnées {symbol_name} récupérées:")
    print(f"Période: {df.index.min().strftime('%Y-%m-%d %H:%M')} à {df.index.max().strftime('%Y-%m-%d %H:%M')}")
    print(f"Nombre de périodes ({INTERVAL}): {len(df)}")
    
    # Calculs selon l'intervalle
    if INTERVAL.endswith('m'):
        minutes_per_period = int(INTERVAL[:-1])
        hours = len(df) * minutes_per_period / 60
        days = hours / 24
        print(f"Nombre d'heures: {hours:.1f}")
        print(f"Nombre de jours: {days:.1f}")
    elif INTERVAL.endswith('h'):
        hours_per_period = int(INTERVAL[:-1])
        hours = len(df) * hours_per_period
        days = hours / 24
        print(f"Nombre d'heures: {hours:.1f}")
        print(f"Nombre de jours: {days:.1f}")
    elif INTERVAL == '1d':
        print(f"Nombre de jours: {len(df)}")
    
    print(f"Prix minimum: ${df['low'].min():.2f}")
    print(f"Prix maximum: ${df['high'].max():.2f}")
    
    return df

# Création du DataFrame Bitcoin
df_btc = create_dataframe(btc_response, "Bitcoin")

print(f"\nPremières lignes Bitcoin (intervalle {INTERVAL}):")
print(df_btc.head())

Configuration:
- Date de fin: Aujourd'hui
- Durée d'analyse: 1 jours
- Intervalle: 1m
- Moyenne mobile: 30 périodes

Récupération des données Bitcoin...
=== Récupération Bitcoin (BTCUSDT) - Intervalle 1m (1 jours) ===
Période: 1 jours jusqu'à aujourd'hui
Batch 1 (BTCUSDT): Récupération depuis 2025-09-13 18:58
Batch 2 (BTCUSDT): Récupération depuis 2025-09-14 11:39
Total de 1440 périodes (1m) récupérées pour BTCUSDT

Données Bitcoin récupérées:
Période: 2025-09-13 16:59 à 2025-09-14 16:58
Nombre de périodes (1m): 1440
Nombre d'heures: 24.0
Nombre de jours: 1.0
Prix minimum: $115141.80
Prix maximum: $116165.19

Premières lignes Bitcoin (intervalle 1m):
                          open       high        low      close     volume  \
timestamp                                                                    
2025-09-13 16:59:00  115341.24  115341.24  115322.00  115334.00    4.30449   
2025-09-13 17:00:00  115333.99  115406.82  115308.92  115322.02  101.53084   
2025-09-13 17:01:00  115322.0

In [2]:
# Graphique Bitcoin avec moyenne mobile, FFT et reconstruction
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# PARAMÈTRE DE RECONSTRUCTION
PERIODE_MAX_RECONSTRUCTION = 3  # Période maximum en heures pour la reconstruction

# Calcul de la moyenne mobile selon le paramètre configuré
df_btc[f'ma_{MA_PERIODE}'] = df_btc['close'].rolling(window=MA_PERIODE).mean()

# Décalage de la moyenne mobile de PERIODE/2 vers la gauche
decalage = MA_PERIODE // 2
df_btc[f'ma_{MA_PERIODE}_shifted'] = df_btc[f'ma_{MA_PERIODE}'].shift(-decalage)

# Calcul de la différence entre le prix et la moyenne mobile décalée
df_btc['difference'] = df_btc['close'] - df_btc[f'ma_{MA_PERIODE}_shifted']

# FFT sur la DIFFÉRENCE (et non sur les prix bruts)
difference_values = df_btc['difference'].dropna().values
fft_values = np.fft.fft(difference_values)
fft_frequencies = np.fft.fftfreq(len(difference_values))

# Magnitude du spectre FFT (seulement la moitié positive)
n = len(difference_values)
fft_magnitude = np.abs(fft_values[:n//2])
fft_freq_positive = fft_frequencies[:n//2]

# Conversion des fréquences en périodes CORRECTE selon l'intervalle
# Déterminer la durée d'une période en minutes
if INTERVAL.endswith('m'):
    minutes_per_period = int(INTERVAL[:-1])
elif INTERVAL.endswith('h'):
    minutes_per_period = int(INTERVAL[:-1]) * 60
elif INTERVAL == '1d':
    minutes_per_period = 24 * 60
else:
    minutes_per_period = 1  # défaut

# Conversion des fréquences en périodes avec protection contre division par zéro
# Seuil minimum pour éviter les divisions par zéro
min_freq_threshold = 1e-10

# Conversion sécurisée
periods_minutes = np.where(
    np.abs(fft_freq_positive) > min_freq_threshold, 
    1 / np.abs(fft_freq_positive) * minutes_per_period, 
    np.inf
)
periods_hours = periods_minutes / 60

# RECONSTRUCTION BASÉE SUR LES PICS FFT
# Filtrer les fréquences selon la période maximum
if INTERVAL.endswith('m'):
    periode_max_heures = PERIODE_MAX_RECONSTRUCTION
    periode_min_heures = 0.1
elif INTERVAL.endswith('h') or INTERVAL == '1d':
    periode_max_heures = PERIODE_MAX_RECONSTRUCTION * 24  # Convertir en heures si jours
    periode_min_heures = 1
else:
    periode_max_heures = PERIODE_MAX_RECONSTRUCTION
    periode_min_heures = 0.1

# Identifier les pics significatifs dans la plage spécifiée
reconstruction_mask = (periods_hours >= periode_min_heures) & (periods_hours <= periode_max_heures)
valid_frequencies = fft_freq_positive[reconstruction_mask]
valid_magnitudes = fft_magnitude[reconstruction_mask]
valid_periods = periods_hours[reconstruction_mask]

# Trouver les pics (maxima locaux)
peak_indices = []
if len(valid_magnitudes) > 2:  # Protection contre les tableaux trop petits
    for i in range(1, len(valid_magnitudes)-1):
        if (valid_magnitudes[i] > valid_magnitudes[i-1] and 
            valid_magnitudes[i] > valid_magnitudes[i+1] and
            valid_magnitudes[i] > np.max(valid_magnitudes) * 0.1):  # Seuil remonté à 10%
            peak_indices.append(i)

# Sélectionner les composantes pour la reconstruction (top pics seulement)
reconstruction_components = []
if peak_indices:
    # Trier par magnitude décroissante
    sorted_peaks = sorted(peak_indices, key=lambda x: valid_magnitudes[x], reverse=True)
    top_peaks = sorted_peaks[:10]  # Revenir au top 10 au lieu de tous les pics
    
    print(f"FFT appliquée sur la DIFFÉRENCE (Prix - MA décalée)")
    print(f"Reconstruction basée sur les {len(top_peaks)} pics les plus importants:")
    
    for idx in top_peaks:  # Utiliser seulement le top 10
        freq = valid_frequencies[idx]
        period = valid_periods[idx]
        magnitude = valid_magnitudes[idx]
        
        # Récupérer la phase de la composante complexe
        # Trouver l'index dans le spectre complet
        full_idx_candidates = np.where(np.abs(fft_freq_positive - freq) < 1e-12)[0]
        if len(full_idx_candidates) > 0:
            full_idx = full_idx_candidates[0]
            phase = np.angle(fft_values[full_idx])
            
            reconstruction_components.append({
                'frequency': freq,
                'magnitude': magnitude,
                'phase': phase,
                'period': period
            })
            print(f"- Période: {period:.2f}h, Magnitude: {magnitude:.0f}")

# Reconstruction de la courbe temporelle
time_indices = np.arange(len(difference_values))
reconstructed_signal = np.zeros(len(difference_values))

for comp in reconstruction_components:
    # Générer la composante sinusoïdale
    component = comp['magnitude'] * np.cos(2 * np.pi * comp['frequency'] * time_indices + comp['phase'])
    reconstructed_signal += component

# Centrer le signal reconstruit sur zéro (sans ajouter la moyenne)
reconstructed_signal = reconstructed_signal - np.mean(reconstructed_signal)

# Création de sous-graphiques avec axes Y secondaires
fig_btc = make_subplots(
    rows=3, cols=1,
    shared_xaxes=False,
    vertical_spacing=0.08,
    row_heights=[0.5, 0.25, 0.25],
    subplot_titles=(
        f'Bitcoin - Prix et Moyenne Mobile {MA_PERIODE} périodes décalée',
        f'Différence (Prix - MA) + Signal Reconstruit FFT TOP 10 (< {PERIODE_MAX_RECONSTRUCTION}h)',
        'Transformée de Fourier sur la Différence'
    ),
    specs=[[{"secondary_y": False}],
           [{"secondary_y": True}],
           [{"secondary_y": False}]]
)

# Graphique principal : Prix et moyenne mobile
fig_btc.add_trace(
    go.Scatter(
        x=df_btc.index,
        y=df_btc['close'],
        mode='lines',
        name='Bitcoin',
        line=dict(color='#f7931a', width=1)
    ),
    row=1, col=1
)

fig_btc.add_trace(
    go.Scatter(
        x=df_btc.index,
        y=df_btc[f'ma_{MA_PERIODE}_shifted'],
        mode='lines',
        name=f'MM {MA_PERIODE} périodes (décalée -{decalage})',
        line=dict(color='blue', width=1),
        opacity=0.9
    ),
    row=1, col=1
)

# Graphique de la différence (axe Y principal)
fig_btc.add_trace(
    go.Scatter(
        x=df_btc.index,
        y=df_btc['difference'],
        mode='lines',
        name='Différence',
        line=dict(color='red', width=1),
        opacity=0.8
    ),
    row=2, col=1, secondary_y=False
)

# Signal reconstruit FFT (axe Y secondaire)
if len(reconstructed_signal) > 0:
    time_axis = df_btc.index[:len(difference_values)]
    fig_btc.add_trace(
        go.Scatter(
            x=time_axis,
            y=reconstructed_signal,
            mode='lines',
            name=f'Signal reconstruit FFT TOP 10 (< {PERIODE_MAX_RECONSTRUCTION}h)',
            line=dict(color='green', width=1),
            opacity=0.9
        ),
        row=2, col=1, secondary_y=True
    )

# Ligne zéro pour référence (axe principal)
fig_btc.add_hline(y=0, line_dash="dash", line_color="gray", opacity=0.5, row=2, col=1)

# Graphique FFT - Spectre de fréquences
if INTERVAL.endswith('m'):
    mask = (periods_hours >= 0.1) & (periods_hours <= 12) & np.isfinite(periods_hours)
    x_label = "Période (heures)"
    x_values = periods_hours[mask]
elif INTERVAL.endswith('h') or INTERVAL == '1d':
    periods_days = periods_hours / 24
    mask = (periods_days >= 0.1) & (periods_days <= DUREE_JOURS/2) & np.isfinite(periods_days)
    x_label = "Période (jours)"
    x_values = periods_days[mask]
else:
    mask = (periods_hours >= 1) & (periods_hours <= 24) & np.isfinite(periods_hours)
    x_label = "Période (heures)"
    x_values = periods_hours[mask]

if len(x_values) > 0:
    fig_btc.add_trace(
        go.Scatter(
            x=x_values,
            y=fft_magnitude[mask],
            mode='lines',
            name='Spectre FFT (différence)',
            line=dict(color='purple', width=1),
            opacity=0.8
        ),
        row=3, col=1
    )

# Marquer les pics utilisés pour la reconstruction
if reconstruction_components:
    peak_periods = [comp['period'] for comp in reconstruction_components]
    peak_magnitudes = [comp['magnitude'] for comp in reconstruction_components]
    
    if INTERVAL.endswith('m'):
        peak_x = peak_periods
    else:
        peak_x = [p/24 for p in peak_periods]  # Convertir en jours si nécessaire
    
    fig_btc.add_trace(
        go.Scatter(
            x=peak_x,
            y=peak_magnitudes,
            mode='markers',
            name=f'Top {len(peak_periods)} pics utilisés',
            marker=dict(color='red', size=8, symbol='circle'),
            showlegend=True
        ),
        row=3, col=1
    )

# Configuration du layout
periode_text = f"jusqu'au {DATE_FIN}" if DATE_FIN else "jusqu'à aujourd'hui"
fig_btc.update_layout(
    title={
        'text': f"Bitcoin - Analyse FFT avec Reconstruction TOP 10 ({DUREE_JOURS} jours {periode_text})",
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 16}
    },
    height=1000,
    showlegend=True,
    legend=dict(
        x=0.01,
        y=0.99,
        bgcolor='rgba(255, 255, 255, 0.8)',
        bordercolor='rgba(0, 0, 0, 0.2)',
        borderwidth=1
    ),
    xaxis_rangeslider_visible=False,
    template="plotly_white",
    margin=dict(l=50, r=50, t=80, b=50)
)

# Configuration des axes
fig_btc.update_yaxes(title_text="Prix (USD)", row=1, col=1)
fig_btc.update_yaxes(title_text="Différence (USD)", row=2, col=1, secondary_y=False)
fig_btc.update_yaxes(title_text="Signal reconstruit", row=2, col=1, secondary_y=True)
fig_btc.update_yaxes(title_text="Magnitude", row=3, col=1)

# Configuration des axes X
fig_btc.update_xaxes(title_text="Date", row=1, col=1)
fig_btc.update_xaxes(title_text="Date", row=2, col=1)
fig_btc.update_xaxes(title_text=x_label, row=3, col=1)

# Affichage
config = {
    'displayModeBar': True,
    'displaylogo': False,
    'modeBarButtonsToRemove': ['pan2d', 'lasso2d']
}

print(f"Paramètre de reconstruction: cycles < {PERIODE_MAX_RECONSTRUCTION}h")
print(f"Période analysée: {DUREE_JOURS} jours ({len(difference_values)} points de différence à {INTERVAL})")
print(f"Signal reconstruit à partir des {len(reconstruction_components)} pics les plus importants dans la DIFFÉRENCE")
print(f"Seuil remonté à 10% du pic principal, limité au top 10")

fig_btc.show(config=config)

FFT appliquée sur la DIFFÉRENCE (Prix - MA décalée)
Reconstruction basée sur les 10 pics les plus importants:
- Période: 0.40h, Magnitude: 9973
- Période: 0.39h, Magnitude: 7640
- Période: 0.52h, Magnitude: 5658
- Période: 1.07h, Magnitude: 5628
- Période: 0.27h, Magnitude: 5557
- Période: 0.94h, Magnitude: 5552
- Période: 0.60h, Magnitude: 5237
- Période: 0.32h, Magnitude: 5127
- Période: 0.41h, Magnitude: 5075
- Période: 0.29h, Magnitude: 4729


/var/folders/v6/137_2s4x7hj5fd5tmtks862r0000gn/T/ipykernel_29043/397430520.py:47: RuntimeWarning: divide by zero encountered in divide
  1 / np.abs(fft_freq_positive) * minutes_per_period,


Paramètre de reconstruction: cycles < 3h
Période analysée: 1 jours (1411 points de différence à 1m)
Signal reconstruit à partir des 10 pics les plus importants dans la DIFFÉRENCE
Seuil remonté à 10% du pic principal, limité au top 10
